In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GINConv
from torch_geometric.utils import scatter
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset

In [2]:
dataset = TUDataset(root='/tmp/nci1', name='NCI1')
dataset = dataset.shuffle()
n_train = len(dataset) // 10 * 9
n_test = len(dataset) - n_train
train_dataset = dataset[:n_train]
test_dataset = dataset[n_train:]

In [3]:
# グラフ同型ネットワークの定義
class MLP(nn.Module):
    def __init__(self, in_d, mid_d, out_d):
        super(MLP, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_d, mid_d),
            nn.BatchNorm1d(mid_d),
            nn.ReLU(),
            nn.Linear(mid_d, out_d),
            nn.BatchNorm1d(out_d),
            nn.ReLU()
        )

    def forward(self, input):
        return self.main(input)


class GIN(torch.nn.Module):
    def __init__(self, in_d, mid_d, out_d):
        super(GIN, self).__init__()

        self.first_layer = MLP(in_d, mid_d, mid_d)
        self.conv1 = GINConv(MLP(mid_d, mid_d, mid_d))
        self.conv2 = GINConv(MLP(mid_d, mid_d, mid_d))
        self.conv3 = GINConv(MLP(mid_d, mid_d, mid_d))
        self.predictor = nn.Sequential(
            nn.Linear(mid_d, mid_d),
            nn.BatchNorm1d(mid_d),
            nn.ReLU(),
            nn.Linear(mid_d, out_d),
        )

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.first_layer(x)
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        graph_embeddings = scatter(x, batch, dim=0, reduce='max')
        res = self.predictor(graph_embeddings)
        return F.log_softmax(res, dim=1)

In [4]:
model = GIN(dataset.num_node_features, 32, dataset.num_classes)

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-4)

In [6]:
def train(epoch):
    model.train()
    loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    for i in range(epoch):
        for data in loader:
            optimizer.zero_grad()
            out = model(data)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()

In [7]:
%time train(500)

CPU times: user 1h 9min 30s, sys: 4.34 s, total: 1h 9min 34s
Wall time: 5min 48s


In [8]:
model.eval()
acc = 0
for data in DataLoader(test_dataset, batch_size=32):
    pred = model(data).max(1)[1]
    acc += pred.eq(data.y).sum().item()
acc / len(test_dataset)

0.8175182481751825